In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
from dl85 import DL85Predictor
import pandas as pd
from sklearn.model_selection import cross_val_score

dataset = np.genfromtxt("datasets/anneal.txt", delimiter=' ')
X, y = dataset[:, 1:], dataset[:, 0]

"""
col1        = np.array([0.0] * 5 + [1.0] * 0 + [0.0] * 0 + [1.0] * 5).reshape(-1,1)
col2        = np.array([0.0] * 1 + [1.0] * 4 + [0.0] * 4 + [1.0] * 1).reshape(-1,1)
X           = np.concatenate( (col1, col2), axis = 1)
y           = np.array([0.0] * 3 + [1.0] * 2 + [0.0] * 1 + [1.0] * 4)
sensitive   = np.array([0.0] * 1 + [1.0] * 4 + [0.0] * 2 + [1.0] * 3)
#X_train,y_train = X, y

#DATAFRAME
df = pd.DataFrame(X)
df['Class'] = y.tolist()
df['Sensitive'] = sensitive.tolist()
df
"""
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)



In [9]:
def descr(tids):
    """
    p0: ∣{𝑥 ∈ 𝐷 ∣ 𝑥.Sensitive = 0, 𝑥.Class = +}∣
    p1: ∣{𝑥 ∈ 𝐷 ∣ 𝑥.Sensitive = 1, 𝑥.Class = +}∣
    n_zero: ∣{𝑥 ∈ 𝐷 ∣ 𝑥.Sensitive = 0}∣
    n_one: ∣{𝑥 ∈ 𝐷 ∣ 𝑥.Sensitive = 1}∣
    """
    p0, p1, n_zero, n_one= 0, 0, 0, 0
    for i in tids:
        if sensitive[i] == 0.0:
            n_zero+=1
            if y[i] == 1.0:
                p0+=1
        elif sensitive[i] == 1.0:
            n_one+=1
            if y[i] == 1.0:
                p1+=1

    if n_one == 0 and n_zero == 0: d = 0
    elif n_zero == 0: d = -(p1/n_one)
    elif n_one == 0: d = p0/n_zero
    else: d = (p0/n_zero) - (p1/n_one)

    #A higher discrimination means that tuples with
    #Sensitive = 1 are less likely to be classified as positive
    return d

def error(tids):
    classes, supports = np.unique(y_train.take(list(tids)), return_counts=True)
    maxindex = np.argmax(supports)

    return sum(supports) - supports[maxindex] + abs(descr(tids)), classes[maxindex]

descr(range(0,len(X_train)))

NameError: name 'sensitive' is not defined

In [ ]:
clf = DL85Predictor(max_depth=2, error_function=error, time_limit=600)

start = time.perf_counter()
print("Model building...")
clf.fit(X_train)
duration = time.perf_counter() - start
print("Model built. Duration of building =", round(duration, 4))
print("Accuracy DL8.5 on training set =", round(clf.accuracy_, 4))
predict = clf.predict(X_test)
print("Average accuracy on test set =", accuracy_score(y_test, predict))

Model building...
Model built. Duration of building = 0.0034
Accuracy DL8.5 on training set = 0.75
Average accuracy on test set = 0.5
